In [1]:
! pip install chromadb langchain ollama

In [2]:
! pip install langchain_ollama

In [3]:
import getpass
import os

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_ollama.chat_models import ChatOllama
from langchain_community.vectorstores import Chroma
from langchain_ollama.embeddings import OllamaEmbeddings
from langchain_community.document_loaders import CSVLoader
from langchain_community.vectorstores import FAISS

In [4]:
def CSVFileLoader(file_path):
    docs = []
    loader = CSVLoader(file_path=file_path)  # Load each CSV file individually
    docs.extend(loader.load())  # Append loaded data to the list
    return docs

In [5]:
llm = ChatOllama(model="llama3.2" )

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100,
    length_function=len,
    is_separator_regex=False,
)

In [6]:
# Combine all loaded documents
all_documents = CSVFileLoader('D:\AI & ML\Projects\Ecommerce-Laptop-Recomonadation\Data\Super_Small_Clean_Laptops.csv')

In [7]:
all_documents

[Document(metadata={'source': 'D:\\AI & ML\\Projects\\Ecommerce-Laptop-Recomonadation\\Data\\Super_Small_Clean_Laptops.csv', 'row': 0}, page_content="LAPTOP NAME: HP Victus AI AMD Ryzen 5 Hexa Core 8645HS - (16 GB/512 GB SSD/Windows 11 Home/4 GB Graphics/NVIDIA GeF...\nPRICE: 58990\nRATING: 4.3\nSPECIFICATIONS: AMD Ryzen 5 Hexa Core Processor\n16 GB DDR5 RAM\nWindows 11 Home Operating System\n512 GB SSD\n39.62 cm (15.6 inch) Display\n1 Year Onsite Warranty\nPRODUCT URL: https://www.flipkart.com/hp-victus-ai-amd-ryzen-5-hexa-core-8645hs-16-gb-512-gb-ssd-windows-11-home-4-graphics-nvidia-geforce-rtx-2050-fb3003ax-gaming-laptop/p/itme1af8ffef0502?pid=COMH8XZFKQRGWMWT&lid=LSTCOMH8XZFKQRGWMWTNDYQSA&marketplace=FLIPKART&q=Laptops&store=6bo%2Fb5g&srno=s_1_1&otracker=search&otracker1=search&fm=organic&iid=en_BrzwIbL2tX2MCd2oXWAUUQ7wj-jDcphIVpLgHADrQM1X35_Y7IlL33NzdBUR8bvtPnJzUv9HU4CCE7DPnHeaYPUFjCTyOHoHZs-Z5_PS_w0%3D&ppt=hp&ppn=homepage&ssid=7n76hozdhc0000001746468761843&qH=48b773c837465a99\nc

In [8]:
splited_documents = text_splitter.split_documents(all_documents)
print(splited_documents)

[Document(metadata={'source': 'D:\\AI & ML\\Projects\\Ecommerce-Laptop-Recomonadation\\Data\\Super_Small_Clean_Laptops.csv', 'row': 0}, page_content='LAPTOP NAME: HP Victus AI AMD Ryzen 5 Hexa Core 8645HS - (16 GB/512 GB SSD/Windows 11 Home/4 GB Graphics/NVIDIA GeF...\nPRICE: 58990\nRATING: 4.3\nSPECIFICATIONS: AMD Ryzen 5 Hexa Core Processor\n16 GB DDR5 RAM\nWindows 11 Home Operating System\n512 GB SSD\n39.62 cm (15.6 inch) Display\n1 Year Onsite Warranty'), Document(metadata={'source': 'D:\\AI & ML\\Projects\\Ecommerce-Laptop-Recomonadation\\Data\\Super_Small_Clean_Laptops.csv', 'row': 0}, page_content='PRODUCT URL:'), Document(metadata={'source': 'D:\\AI & ML\\Projects\\Ecommerce-Laptop-Recomonadation\\Data\\Super_Small_Clean_Laptops.csv', 'row': 0}, page_content='https://www.flipkart.com/hp-victus-ai-amd-ryzen-5-hexa-core-8645hs-16-gb-512-gb-ssd-windows-11-home-4-graphics-nvidia-geforce-rtx-2050-fb3003ax-gaming-laptop/p/itme1af8ffef0502?pid=COMH8XZFKQRGWMWT&lid=LSTCOMH8XZFKQRGWMWTN

In [9]:
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS

In [10]:
embeddings  = OllamaEmbeddings(
  model='mxbai-embed-large'
)
persist_directory = "./FAISS"

vectorstore = FAISS.from_documents(
    documents=splited_documents,  # Ensure splited_documents contains valid data
    embedding=embeddings
)

print("✅ Data successfully stored in FAISS!")

✅ Data successfully stored in FAISS!


In [11]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.chat_models import ChatOllama

# Initialize your retriever from the FAISS vector store
retriever = vectorstore.as_retriever()

# Initialize your local language model
llm = ChatOllama(model="llama3.2")

# Define the system prompt with the required {context} variable
system_prompt = (
    "Use the given context to answer the question. "
    "If you don't know the answer, say you don't know. "
    "Keep the answer concise and relevant.\n\n"
    "{context}"
)

# Create the prompt template
prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}")
])

# Create the document chain
document_chain = create_stuff_documents_chain(llm, prompt)

# Create the retrieval chain
retrieval_chain = create_retrieval_chain(retriever, document_chain)

C:\Users\gaura\AppData\Local\Temp\ipykernel_13484\4101668005.py:10: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(model="llama3.2")


In [12]:
# Define your query
query = "i want to buy mobile phone please suggest me ?"

# Invoke the retrieval chain with the query
response = retrieval_chain.invoke({"input": query})

# Print the generated answer
print("Answer:", response["answer"])


Answer: I don't see any information about mobile phones in the provided context. The links seem to be related to laptops from Samsung, Galaxy Book4 series. If you're looking for a mobile phone suggestion, I'd need more context or information about your preferences (e.g., budget, brand, features). Would you like me to try and help with that?


In [ ]:
from langchain.chains import ConversationalRetrievalChain
from langchain_core.messages import HumanMessage, AIMessage

# Initialize chat history
chat_history = []

# Define your initial query
query = "Give me best laptops for remote work?"

# Invoke the retrieval chain with the query and chat history
response = retrieval_chain.invoke({"input": query, "chat_history": chat_history})

# Print the generated answer
print("Answer:", response["answer"])

# Update chat history with the latest interaction
chat_history.extend([
    HumanMessage(content=query),
    AIMessage(content=response["answer"])
])

# Define a follow-up question
follow_up_query = "Can you suggest different laptops?"

# Invoke the retrieval chain with the follow-up question and updated chat history
follow_up_response = retrieval_chain.invoke({"input": follow_up_query, "chat_history": chat_history})

# Print the follow-up answer
print("Follow-Up Answer:", follow_up_response["answer"])

# Update chat history with the follow-up interaction
chat_history.extend([
    HumanMessage(content=follow_up_query),
    AIMessage(content=follow_up_response["answer"])
])
